<a href="https://colab.research.google.com/github/artemkush1/ML_predict_CS-GO_matches/blob/master/data_collect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pprint as pp
import requests
import datetime
from bs4 import BeautifulSoup
from re import sub
import re
import numpy as np
import pandas as pd

pr = pp.PrettyPrinter()
prefix = 'https://www.hltv.org'
max_rank = 1000

In [0]:
def get_parsed_page(url):
    # This fixes a blocked by cloudflare error i've encountered
    headers = {
        "referer": "https://www.hltv.org/stats",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }
    return BeautifulSoup(requests.get(url, headers=headers).text, "lxml")

In [0]:
results = get_parsed_page('https://www.hltv.org/results')

# list of matches
matches = results.find('div', {"class": "results-all", 'data-zonedgrouping-group-classes': "results-sublist"}).\
    find_all('div', {"class": "result-con"})

current_match = matches[0]
match_url = prefix + current_match.find('a', {"class": "a-reset"})['href']  # to DB
match_page = get_parsed_page(match_url)
event_page = get_parsed_page(prefix + match_page.find('div', {'class': 'event'}).find('a')['href'])
teams_pages = [get_parsed_page(i) for i in get_teams_url(match_page)]
team1_url, team2_url = get_teams_url(match_page)  # to DB

# Good functions

In [88]:
# 2 urls pf teams
def get_teams_url(match_page):
    return [prefix + match_page.find_all('div', {"class": "team"})[0].find('a')['href'],
            prefix + match_page.find_all('div', {"class": "team"})[1].find('a')['href']]


# get score: expect 2 params
def get_score(current_match):
    tmp = [int(i.text) for i in current_match.find('td', {'class': 'result-score'}).find_all('span')]
    if len(tmp) != 2:
        print("Problems in get_score()")
    return tmp


# get total_maps: expect 1 param
def get_total_maps(current_match):
    tmp = current_match.find('td', {'class': 'star-cell'}).find('div', {'class': 'map-text'}).text
    if not tmp:
        print("Problems in get_total_maps()")
    return tmp


# get star_cell: expect 1 param
def get_star_cell(current_match):
    try:
        tmp = len(current_match.find('td', {'class': 'star-cell'}).find_all('i', {'class': 'star'}))
        if not tmp:
            print("Problems in get_star_cell()")
        return tmp
    except Exception:
        print('None stars')
        return 0


# get history_h2h: expect 3 params
def get_history_h2h(match_page):
    tmp = [int(i.find('div', {'class': 'bold'}).text) for i in match_page.find_all('div', {'class': 'flexbox-center'})]
    if len(tmp) != 3:
        print("Problems in get_history_h2h()")
    return tmp


# get teams_rank: expect 1 param
def get_team_rank(team):
    try:
        tmp = int(team.find('div', {'class': 'profile-team-stats-container'}).
                  find_all('span', {'class': "right"})[0].text[1:])
        if not tmp:
            print("Problems in get_team_rank()")
        return tmp
    except:
        return max_rank


# get teams_top30_for_core: expect 1 param
def get_top30_for_core(team):
    try:
        tmp = int(team.find('div', {'class': 'profile-team-stats-container'}).
                  find_all('span', {'class': "right"})[1].text)
        if not tmp:
            print("Problems in get_team_rank()")
        return tmp
    except Exception as e:
        return 0


# get average_ages: expect 1 param
def get_ave_age(team):
    try:
        tmp = float(team.find('div', {'class': 'profile-team-stats-container'}).
                     find_all('span', {'class': "right"})[2].text)
        if not tmp:
            print("Problems in get_ave_age()")
        return tmp
    except Exception as e:
        return 0


# get prize_pool: expect 1 param
def get_prize_pool(event_page):
    try:
        return int(sub(r'[^\d.]', '', event_page.find('td', {'class': 'prizepool'}).text))
    except:
        return 0

# get teams_on_tour: expect 1 param
def get_teams_on_tour(event_page):
    tmp = int(re.findall(r"[-+]?\d*\.\d+|\d+", event_page.find('td', {'class': 'teamsNumber'}).text)[0])
    if tmp:
        return tmp
    else:
        return 0


# get type_of_tour: expect 1 param
def get_type_tour(event_page):
    tmp = event_page.find('td', {'class': 'location'}).find('span').text
    if tmp:
        return [i for i in re.split('\(|\)| ', tmp) if i is not ''][-1]
    else:
        return 'None'


def get_players_from_team(team):
    return team.find('div', {'class': 'bodyshot-team'}).find_all('a')


# expect 1 param
def get_player_age(player):
    tmp = re.findall(r"[-+]?\d*\.\d+|\d+", (player.find('div', {'class': 'summaryPlayerAge'}).text))[0]
    if tmp:
        return tmp
    else:
        return 0


# expect 2 * 5 params
def get_players_from_match_url(match):
    tmp = [[prefix + '/stats/players' + (i.find('a')['href'].split('/player')[-1])
             for i in k.find_all('td', {'class': 'player'})][:5]
            for k in match.find_all('div', {'class': 'lineup standard-box'})]
    if len(tmp) != 2 or  len(tmp[0]) != 5 or len(tmp[1]) != 5:
        print("Problems in get_players_from_match_url()")
    return tmp


# expect 19 params
def get_player_many_stats(player):
    tmp = player.find('div', {'class': 'statistics'}).find_all('div', {'class': 'stats-row'})
    res = []
    for i in tmp:
        tmp1 = i.find_all('span')[-1].text
        if tmp1[-1] == '%':
            tmp1 = float(tmp1[:-1]) / 100
        elif tmp1[-1] == '-':
            tmp1 = '0'
        res.append(tmp1)
    tmp = player.find('div', {'class': 'featured-ratings-container'}).find_all('div', {'class': 'col-custom'})
    for i in tmp:
        tmp1 = i.find('div', {'class': 'rating-value'}).text
        if tmp1[-1] == '%':
            tmp1 = float(tmp1[:-1]) / 100
        elif tmp1[-1] == '-':
            tmp1 = '0'
        res.append(tmp1)
    if len(res) != 19:
        print("Problems in get_player_many_stats()")
    return res


# get lats 5 matches for both teams: expect 30 params
def get_last5_matches_v1_2teams(match):
    def get_url_opponent(url1, i):
        return prefix + url1.find_all('tr', {'class': 'table'})[i].find('a', {'class': 'text-ellipsis'})['href']

    def get_type_match(url1, i):
        return url1.find_all('tr', {'class': 'table'})[i].find('a', {'data-link-tracking-page': 'Matchpage'}).text

    def get_score_array(url1, i):
        return [int(k) for k in
                url1.find_all('tr', {'class': 'table'})[i].find('td', {'class': 'spoiler'}).text.split(' - ')]

    def get_all(table):
        for i in range(5):
            try:
                type_ = get_type_match(table, i)
                rank_ = get_team_rank(get_parsed_page(get_url_opponent(table, i)))
                score1, score2 = get_score_array(table, i)
                types.append(type_)
                teams_rank.append(rank_)
                if score1 > score2:
                    scores.append(1)
                elif score1 < score2:
                    scores.append(-1)
                else:
                    scores.append(0)
            except Exception as e:
                print(e)
                types.append('None')
                teams_rank.append('0')
                scores.append(0)
    types = []
    teams_rank = []
    scores = []
    tables = (match_page.find('div', {'class': 'past-matches'}).
             find_all('div', {'class': 'half-width'}))
    last5_matches_2teams = []
    get_all(tables[0])
    get_all(tables[1])
    last5_matches_2teams = [*types, *teams_rank, *scores]
    if len(last5_matches_2teams) != 30:
        print('Problems in get_last5_matches_v1_2teams()')
    return last5_matches_2teams


get_last5_matches_v1_2teams(match_page)
#players = get_players_from_match_url(match_page)
#print(players[0][0])
#get_player_age(get_parsed_page('https://www.hltv.org/stats/players/17939/leaf'))
# get_players_many_stats(get_parsed_page('https://www.hltv.org/stats/players/17939/leaf'))
get_last5_matches_v1_2teams(match_page)




['Best of 3',
 'Best of 3',
 'Best of 3',
 'Best of 3',
 'Mirage',
 'Best of 3',
 'Inferno',
 'Best of 3',
 'Overpass',
 'Best of 3',
 50,
 107,
 17,
 47,
 17,
 47,
 50,
 85,
 32,
 26,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 -1]

# Bad functions

In [94]:
# <=============== FORMATTING ===============> #
import time


def create_vec_features(current_match):
    match_url = prefix + current_match.find('a', {"class": "a-reset"})['href']  # to DB
    print(match_url)
    match_page = get_parsed_page(match_url)
    event_page = get_parsed_page(prefix + match_page.find('div', {'class': 'event'}).find('a')['href'])
    teams_pages = [get_parsed_page(i) for i in get_teams_url(match_page)]
    team1_url, team2_url = get_teams_url(match_page)  # to DB

    score_arr = get_score(current_match)
    counter_maps = get_total_maps(current_match)
    stars = get_star_cell(current_match)
    history_meetings = get_history_h2h(match_page)
    rank1, rank2 = get_team_rank(teams_pages[0]), get_team_rank(teams_pages[1])
    top30forcore1, top30forcore2 = get_top30_for_core(teams_pages[0]), get_top30_for_core(teams_pages[1])
    av_age1, av_age2 = get_ave_age(teams_pages[0]), get_ave_age(teams_pages[1])
    prize_pool = get_prize_pool(event_page)
    teamsontour = get_teams_on_tour(event_page)
    typetour = get_type_tour(event_page)

    players_2Darray = get_players_from_match_url(match_page)
    players_stats = []
    players_years = []
    for i in players_2Darray:  # 2
        for k in i:  # 10
            # print(k)
            player_page = get_parsed_page(k)

            time.sleep(0.1)  # fix problems with loading pages

            players_stats += get_players_many_stats(player_page)  # 19
            players_years.append(get_player_age(player_page))  # 1
    last5_matches = get_last5_matches_v1_2teams(match_page)
    return np.array([*score_arr,
                     counter_maps,
                     prize_pool,
                     teamsontour,
                     typetour,
                     stars,
                     *history_meetings,
                     rank1, rank2,
                     top30forcore1, top30forcore2,
                     av_age1, av_age2,
                     *players_stats,
                     *players_years,
                     *last5_matches])








header = ['score1', 'score2',
          'match_type',
          'prize_pool',
          'teams_on_tour',
          'type_tour',
          'stars',
          'history_team1', 'history_draw', 'history_team2',
          'rank1', 'rank2',
          'top30forcore1', 'top30forcore2',
          'av_age1', 'av_age2',
          *[f'stat_{i}_{k}_{j}' for j in range(2) for k in range(5) for i in range(19)],
          *[f'age_{i}_{k}' for k in range(2) for i in range(5)],
          *[f'prev_type_match_{i}_{k}' for k in range(2) for i in range(5)],
          *[f'prev_match_rank_{i}_{k}' for k in range(2) for i in range(5)],
          *[f'prev_match_score_{i}_{k}' for k in range(2) for i in range(5)]
          ]




len_head = len(header)
data = [np.array(header)]
print(len_head)


for i in range(10):
    if i == 0:
        results = get_parsed_page('https://www.hltv.org/results')
    else:
        results = get_parsed_page(f'https://www.hltv.org/results?offset={i}00')
    # list of matches
    matches = results.find('div', {"class": "results-all", 'data-zonedgrouping-group-classes': "results-sublist"}). \
        find_all('div', {"class": "result-con"})
    for k in range(len(matches)):
        print(i*100 + k)
        try:
            tmp = create_vec_features(matches[k])
            print(len(tmp))
            if len_head == len(tmp):
                data.append(tmp)
        except Exception as e:
            print(e)
    if k // 10 == 0:
        data_save = np.array(data)
        df = pd.DataFrame(data=data_save[1:,:], columns=data_save[0, :])
        df.to_csv('df.csv')


# https://www.hltv.org/results?offset=100





246
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27

KeyboardInterrupt: ignored

In [93]:
from google.colab import drive
drive.mount('/content/drive')

data_save = np.array(data)
df = pd.DataFrame(data=data_save[1:,:], columns=data_save[0, :])
df.to_csv('/content/drive/My Drive/Colab Notebooks/cs-go_matches/df.csv')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def get_parsed_page(url):
    # This fixes a blocked by cloudflare error i've encountered
    headers = {
        "referer": "https://www.hltv.org/stats",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }
    return BeautifulSoup(requests.get(url, headers=headers).text, "lxml")